<a href="https://colab.research.google.com/github/abosedealli/Stockprediction/blob/main/LINEAR%20REGRESSION%20stock_dataset_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
!pip install tensorflow
import tensorflow as tf
import random

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Importing the libraries to Predict the closing price stock price 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.callbacks import History 
import seaborn as sns
import keras
from keras.callbacks import History 
from sklearn.metrics import mean_squared_error
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


In [ ]:
sns.set_style('whitegrid')
from datetime import datetime

## Stocks to use for analysis

---

Apple dataset from inception till till 9/1/2023 in US currency except date and volume.



In [ ]:
data = pd.read_csv("/content/appledataset.csv")


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10564 entries, 0 to 10563
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       10564 non-null  object 
 1   High       10564 non-null  float64
 2   Low        10564 non-null  float64
 3   Open       10564 non-null  float64
 4   Close      10564 non-null  float64
 5   Volume     10564 non-null  int64  
 6   Adj Close  10564 non-null  float64
dtypes: float64(5), int64(1), object(1)
memory usage: 577.8+ KB


In [ ]:
data.set_index(pd.DatetimeIndex(data['Date']), inplace=True)

In [ ]:
data

,Date,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,,
1980-12-12,12/12/1980,0.128906,0.128348,0.128348,0.128348,469033600,0.100039
1980-12-15,15/12/1980,0.122210,0.121652,0.122210,0.121652,175884800,0.094820
1980-12-16,16/12/1980,0.113281,0.112723,0.113281,0.112723,105728000,0.087861
1980-12-17,17/12/1980,0.116071,0.115513,0.115513,0.115513,86441600,0.090035
1980-12-18,18/12/1980,0.119420,0.118862,0.118862,0.118862,73449600,0.092646
...,...,...,...,...,...,...,...
2022-10-28,28/10/2022,157.500000,147.820007,148.199997,155.740005,164762400,155.740005
2022-10-31,31/10/2022,154.240005,151.919998,153.160004,153.339996,97943200,153.339996
2022-01-11,1/11/2022,155.449997,149.130005,155.080002,150.649994,80379300,150.649994


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10564 entries, 1980-12-12 to 2022-03-11
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       10564 non-null  object 
 1   High       10564 non-null  float64
 2   Low        10564 non-null  float64
 3   Open       10564 non-null  float64
 4   Close      10564 non-null  float64
 5   Volume     10564 non-null  int64  
 6   Adj Close  10564 non-null  float64
dtypes: float64(5), int64(1), object(1)
memory usage: 660.2+ KB


In [ ]:
apple= data.assign(**{'average': lambda df: df.loc[:,['Open','High','Low','Close']].mean(axis=1), 
                    'EMA20': lambda df: df['average'].ewm(span=20, adjust=False).mean(), 
                    'EMA5': lambda df: df['average'].ewm(span=5, adjust=False).mean(), 
                    'dist_EMA20': lambda df: (df['average'] - df['EMA20'])/df['EMA20']*100, 
                    'dist_EMA5': lambda df: (df['average'] - df['EMA5'])/df['EMA5']*100})
apple.head()

,Date,High,Low,Open,Close,Volume,Adj Close,average,EMA20,EMA5,dist_EMA20,dist_EMA5
Date,,,,,,,,,,,,
1980-12-12,12/12/1980,0.128906,0.128348,0.128348,0.128348,469033600,0.100039,0.128487,0.128487,0.128487,0.000000,0.000000
1980-12-15,15/12/1980,0.122210,0.121652,0.122210,0.121652,175884800,0.094820,0.121931,0.127863,0.126302,-4.639389,-3.460749
1980-12-16,16/12/1980,0.113281,0.112723,0.113281,0.112723,105728000,0.087861,0.113002,0.126448,0.121869,-10.633428,-7.275591
1980-12-17,17/12/1980,0.116071,0.115513,0.115513,0.115513,86441600,0.090035,0.115652,0.125420,0.119797,-7.787547,-3.459289
1980-12-18,18/12/1980,0.119420,0.118862,0.118862,0.118862,73449600,0.092646,0.119002,0.124808,0.119532,-4.652620,-0.443457


In [ ]:
apple.columns

Index(['Date', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close',
       'average', 'EMA20', 'EMA5', 'dist_EMA20', 'dist_EMA5'],
      dtype='object')

In [ ]:
apple

,Date,High,Low,Open,Close,Volume,Adj Close,average,EMA20,EMA5,dist_EMA20,dist_EMA5
Date,,,,,,,,,,,,
1980-12-12,12/12/1980,0.128906,0.128348,0.128348,0.128348,469033600,0.100039,0.128487,0.128487,0.128487,0.000000,0.000000
1980-12-15,15/12/1980,0.122210,0.121652,0.122210,0.121652,175884800,0.094820,0.121931,0.127863,0.126302,-4.639389,-3.460749
1980-12-16,16/12/1980,0.113281,0.112723,0.113281,0.112723,105728000,0.087861,0.113002,0.126448,0.121869,-10.633428,-7.275591
1980-12-17,17/12/1980,0.116071,0.115513,0.115513,0.115513,86441600,0.090035,0.115652,0.125420,0.119797,-7.787547,-3.459289
1980-12-18,18/12/1980,0.119420,0.118862,0.118862,0.118862,73449600,0.092646,0.119002,0.124808,0.119532,-4.652620,-0.443457
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-28,28/10/2022,157.500000,147.820007,148.199997,155.740005,164762400,155.740005,152.315002,146.697544,149.153160,3.829279,2.119863
2022-10-31,31/10/2022,154.240005,151.919998,153.160004,153.339996,97943200,153.339996,153.165001,147.313492,150.490440,3.972147,1.777230
2022-01-11,1/11/2022,155.449997,149.130005,155.080002,150.649994,80379300,150.649994,152.577499,147.814826,151.186127,3.222054,0.920304


In [ ]:
apple_model_data=apple[['Adj Close', 'High', 'EMA20', 'Close']]
# apple.columns

In [ ]:
apple_model_data.head()

,Adj Close,High,EMA20,Close
Date,,,,
1980-12-12,0.100039,0.128906,0.128487,0.128348
1980-12-15,0.094820,0.122210,0.127863,0.121652
1980-12-16,0.087861,0.113281,0.126448,0.112723
1980-12-17,0.090035,0.116071,0.125420,0.115513
1980-12-18,0.092646,0.119420,0.124808,0.118862


In [ ]:
X = apple_model_data[['Adj Close', 'EMA20', 'High']]
Y = apple_model_data['Close']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state = 99)


In [ ]:
X_train.head()

,Adj Close,EMA20,High
Date,,,
2021-09-21,142.630753,148.960193,144.600006
2019-09-04,48.424793,47.576642,50.712502
2006-02-27,2.164768,2.534798,2.575714
2016-01-19,22.159309,25.697598,24.662500
1988-10-28,0.270847,0.357266,0.352679


In [ ]:
print(X_test.describe())

         Adj Close        EMA20         High
count  2113.000000  2113.000000  2113.000000
mean     14.423319    14.983416    15.211244
std      32.627081    32.874478    33.259908
min       0.040016     0.055964     0.051897
25%       0.243135     0.292652     0.303571
50%       0.404522     0.482208     0.493304
75%      12.131637    13.935661    14.364286
max     178.449570   174.142804   179.610001


In [ ]:
print(X_train.describe())

         Adj Close        EMA20         High
count  8451.000000  8451.000000  8451.000000
mean     15.602995    16.100410    16.431862
std      34.443980    34.403238    35.134094
min       0.038276     0.055824     0.049665
25%       0.235778     0.283287     0.290179
50%       0.399188     0.478133     0.488839
75%      13.883844    15.833619    16.255714
max     181.259918   174.927371   182.940002


In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  mape = np.mean(np.abs((y_true - y_pred) / y_true))*100
  return mape

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(x_train, y_train)

# Make the prediction
y_pred = reg.predict(x_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape= mean_absolute_percentage_error(y_test, y_pred)
print('Mean abosulte percentage error:', mape)

Mean abosulte percentage error: 27.29246479998685


In [ ]:
mae = mean_absolute_error(y_test, y_pred)
print('Mean Absolute Error:', mae)
print('Mean Squared Error:', mse)
print('R Square value:', r2)

Mean Absolute Error: 0.19816830697828264
Mean Squared Error: 0.13508599415382325
R Square value: 0.9998752641190555


Linear regression is not a good model for time series data thus we will use LSTM.